# Mini-sdAIgen
A lightweight, focused downloader for ComfyUI workflows on Kaggle/Colab.

### Instructions
1. Run the **Setup** cell to clone the repository (sparse checkout), install dependencies, **and install ComfyUI**.
2. Run the **Widgets** cell to configure your API tokens and Model URLs.
3. Run the **Download** cell to start downloading.
4. Run the **Launch** cell to start ComfyUI.

In [ ]:
# 1. Setup Environment
import os
import sys
import shutil
import subprocess

REPO_NAME = 'mini-sdAIgen'

# Logic: Clone if missing, Pull if exists
if not os.path.exists(REPO_NAME):
    print(f"Cloning {REPO_NAME} repository (Sparse Checkout)...")
    
    # 1. Prepare temp dir
    temp_dir = "temp_clone"
    os.makedirs(temp_dir, exist_ok=True)
    original_dir = os.getcwd()
    os.chdir(temp_dir)
    
    try:
        # 2. Git Sparse Checkout sequence
        !git init
        !git remote add origin https://github.com/Faturrachman-dev/notebook.git
        !git config core.sparseCheckout true
        !echo "mini-sdAIgen/" >> .git/info/sparse-checkout
        !git pull --depth 1 origin main
        
        # 3. Move target folder out
        os.chdir(original_dir)
        if os.path.exists(f"{temp_dir}/{REPO_NAME}"):
            # Move the folder to root
            if os.path.exists(REPO_NAME):
                 shutil.rmtree(REPO_NAME)
            shutil.move(f"{temp_dir}/{REPO_NAME}", ".")
            print("Clone successful.")
        else:
            print(f"Error: {REPO_NAME} folder not found in remote repo. Please check repo structure.") 
            
    except Exception as e:
        os.chdir(original_dir)
        print(f"Clone failed: {e}")
    finally:
        # 4. Cleanup
        if os.path.exists(temp_dir):
            if sys.platform == 'win32':
                !rmdir /S /Q temp_clone
            else:
                !rm -rf temp_clone
else:
    print(f"{REPO_NAME} exists. Pulling latest changes...")
    # Note: Since it's a moved folder, it might not be a valid git root itself anymore depending on how we moved it.
    # The previous logic MOVED the folder out of the temp git repo.
    # So 'mini-sdAIgen' folder is NOT a git repo on the disk.
    # TO FIX: We should just DELETE and RE-CLONE to be safe and simple.
    print("Updating: Deleting old version and re-cloning...")
    shutil.rmtree(REPO_NAME)
    # Recursively call this logic? No, just copy-paste or set flag.
    # Re-running the cell will now trigger the 'if not exists' block above.
    print(">> PLEASE RE-RUN THIS CELL IMMEDIATELY to clone the fresh version. <<")

# Ensure repo is in path
repo_path = os.path.abspath(REPO_NAME)
if repo_path not in sys.path and os.path.exists(repo_path):
    sys.path.append(repo_path)

# Run setup script (Handles Deps, ComfyUI, Manager, SQLAlchemy, PyNgrok)
if os.path.exists(f"{REPO_NAME}/setup.py"):
    !python {REPO_NAME}/setup.py
else:
    print("setup.py not found (yet). If you just deleted the folder, please re-run this cell.")

In [ ]:
# 2. Configure Widgets
import launch
from importlib import reload
reload(launch) # Ensure we have the latest version
launch.show_widgets()

In [ ]:
# 3. Start Download
launch.run_download()

In [ ]:
# 4. Launch ComfyUI
import json
import os
from pyngrok import ngrok, conf

# Defines
COMFY_ROOT = '/root/ComfyUI' if os.path.exists('/root/ComfyUI') else 'ComfyUI'
SETTINGS_PATH = 'settings.json'

if os.path.exists(os.path.join(COMFY_ROOT, 'main.py')):
    # 1. Setup Ngrok
    if os.path.exists(SETTINGS_PATH):
        try:
            with open(SETTINGS_PATH, 'r') as f:
                settings = json.load(f)
            ngrok_token = settings.get('ngrok_token', '')
            
            if ngrok_token:
                print("Initializing Ngrok Tunnel...")
                conf.get_default().auth_token = ngrok_token
                public_url = ngrok.connect(8188).public_url
                print(f"\n>>> ComfyUI Public URL: {public_url} <<<\n")
            else:
                print("No Ngrok token found. Local access only (127.0.0.1:8188).")
        except Exception as e:
            print(f"Ngrok Setup Error: {e}")

    # 2. Launch ComfyUI directly (Stream Output)
    print("Starting ComfyUI...\n")
    # Using !python allows Jupyter to stream stdout immediately
    !python {COMFY_ROOT}/main.py --listen --port 8188
else:
    print(f"Error: ComfyUI main.py not found at {COMFY_ROOT}")